# THE BATTLE OF NEIGHBORHOODS

We will see all the major venues where Thai restaurants are in mass and which will be suitable place for estabilishing a Thai restaurant 

# Data

For this project we need the following data:

1. Toronto City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.
2. Indian resturants in each neighborhood of new york city.


# Methodology

1. We begin by collecting the Toronto city data from "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
2. We will find all venues for each neighborhood using FourSquare API.
3. We will then filter out all Thai Restuarant venues.
4. We will cluster the restaurants according to the neighbourhood.
5. We will analyse each cluster to find out which area is having less number of Thai restaurant
6. We will visualize the neighborhoods using python's Folium library.

**Importing Libraries**

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from geopy.geocoders import Nominatim

In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

In [3]:
!pip install folium
import folium

In [4]:
!pip install geocoder
import geocoder

**Get the toronto data of neighbourhood and borough**

In [5]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [6]:
import lxml

In [7]:
df = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
df.dropna(subset=['Borough'], inplace=True)

In [10]:
n_empty_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [11]:
df_pc =pd.DataFrame( df.groupby(['Postal code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_pc.reset_index(inplace=True)
df_pc.head(5)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df.dropna(subset=['Borough'], inplace=True)

In [13]:
n_empty_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [14]:
print('The shape of the dataset is:',df_pc.shape)

The shape of the dataset is: (103, 3)


**Convert the dataframe into csv**

In [15]:
df_pc.to_csv('Toronto_Postcodes.csv')

**Get the data of longitude and latitude for Toronto city** 

In [16]:
url_csv = 'http://cocl.us/Geospatial_data'
df_cor = pd.read_csv(url_csv)
df_cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df_cor.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postal code': 'PostalCode'}, inplace=True)

**Merge both csv files to get the final data**

In [19]:
df_neighborhoods_cor = pd.merge(df_neighborhoods, df_cor, on='PostalCode')
df_neighborhoods_cor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
df_neighborhoods_cor[(df_neighborhoods_cor['PostalCode']=='M5G') |
                             (df_neighborhoods_cor['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [21]:
df_neighborhoods_cor.to_csv('Toronto_Postcodes_2.csv')

In [23]:
df = pd.read_csv('Toronto_Postcodes_2.csv',index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [25]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [26]:
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [27]:
df_tor = df[df['Borough'].str.contains('Toronto')]
df_tor.reset_index(inplace=True)
df_tor.drop('index', axis=1, inplace=True)
df_tor.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [28]:
print(df_tor.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [30]:
boroughs = df_tor['Borough'].unique().tolist()

**To find coordinates of Toronto city**

In [31]:
lat_toronto = df_tor['Latitude'].mean()
lon_toronto = df_tor['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [32]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

**Map of Toronto city**

In [33]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], 
                                           df_tor['Longitude'],
                                           df_tor['Borough'], 
                                           df_tor['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

**Now using Foursquare API**

In [34]:
CLIENT_ID = 'NQGVIMAMEKH4BWCE1LD4EI0FWZ2N1CIJHWXXF43CSUQ5JMC4' # your Foursquare ID
CLIENT_SECRET = '23BDSTBRLWCANYZRTHLIV0EAUCS0LTJ3BPIB4GU4AAFSOJK0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                latitudes=df_tor['Latitude'],
                                longitudes=df_tor['Longitude'])

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / 

In [38]:
toronto_venues.shape

(1612, 7)

In [39]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,18,18,18,18,18,18
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,18,18,18,18,18,18
Central Bay Street,63,63,63,63,63,63


**Getting all venue category**

In [40]:
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Diner',
       'Bubble Tea Shop', 'Furniture / Home Store', 'Grocery Store',
       'Spa', 'Coffee Shop', 'Bakery', 'Caribbean Restaurant', 'Café',
       'Indian Restaurant', 'Japanese Restaurant', 'Lounge',
       'Frozen Yogurt Shop', 'American Restaurant', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Park', 'Pet Store', 'Steakhouse', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Fish Market', 'Gay Bar',
       'Seafood Restaurant', 'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Stationery Store', 'Wine Bar',
       'Thai Restaurant', 'Coworking Space', 'Latin American Restaurant',
       'Gastr

**Searching for Thai restaurant**

In [41]:
"Thai Restaurant" in toronto_venues['Venue Category'].unique()

True

In [42]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1612, 232)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped.head()

(39, 232)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.055556
3,CN Tower / King and Spadina / Railway Lands / ...,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.015873,0.0,0.0,0.0,0.0,0.0,0.015873


In [44]:
thai = to_grouped[["Neighborhoods","Thai Restaurant"]]

In [45]:
thai.head()

,Neighborhoods,Thai Restaurant
0,Berczy Park,0.017544
1,Brockton / Parkdale Village / Exhibition Place,0.000000
2,Business reply mail Processing CentrE,0.000000
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000
4,Central Bay Street,0.015873


**Now using clustering we will cluster the Thai restaurants according to neighbourhoods**

In [46]:
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = thai.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [47]:
to_merged = thai.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [48]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head(5)

,Neighborhood,Thai Restaurant,Cluster Labels
0,Berczy Park,0.017544,1
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0
2,Business reply mail Processing CentrE,0.000000,0
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0
4,Central Bay Street,0.015873,1


In [49]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1612, 9)


,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017544,1,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017544,1,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017544,1,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017544,1,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017544,1,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [57]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,Kensington Market / Chinatown / Grange Park,0.0,0,43.653206,-79.400049,Kanto,43.652167,-79.404843,Filipino Restaurant
34,The Annex / North Midtown / Yorkville,0.0,0,43.672710,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
33,Summerhill West / Rathnelly / South Hill / For...,0.0,0,43.686412,-79.400049,Kiva's,43.687984,-79.394715,Bagel Shop
33,Summerhill West / Rathnelly / South Hill / For...,0.0,0,43.686412,-79.400049,TTC Stop #,43.685826,-79.404981,Light Rail Station
34,The Annex / North Midtown / Yorkville,0.0,0,43.672710,-79.405678,LCBO,43.675344,-79.405327,Liquor Store


**The clustered map using folium library**

In [51]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Thai Restaurant') ]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
31,Stn A PO Boxes,0.010417,0,43.646435,-79.374846,Sukhothai,43.648487,-79.374547,Thai Restaurant
13,"Garden District, Ryerson",0.010000,0,43.657162,-79.378937,Salad King,43.657601,-79.381620,Thai Restaurant


In [53]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Thai Restaurant') ]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,St. James Town / Cabbagetown,0.023256,1,43.667967,-79.367675,Thai Room - Cabbagetown,43.664159,-79.368189,Thai Restaurant
32,Studio District,0.025000,1,43.659526,-79.340923,EAT BKK Thai Kitchen,43.660450,-79.343113,Thai Restaurant
0,Berczy Park,0.017544,1,43.644771,-79.373306,Sukhothai,43.648487,-79.374547,Thai Restaurant
6,Church and Wellesley,0.013889,1,43.665860,-79.383160,Si Lom,43.665010,-79.380683,Thai Restaurant
4,Central Bay Street,0.015873,1,43.657952,-79.387383,Thai Express,43.661630,-79.387340,Thai Restaurant
8,Davisville,0.030303,1,43.704324,-79.388790,Thai Spicy House,43.701962,-79.387513,Thai Restaurant
25,Richmond / Adelaide / King,0.031915,1,43.650571,-79.384568,Pai,43.647923,-79.388579,Thai Restaurant
11,First Canadian Place / Underground city,0.020000,1,43.648429,-79.382280,Ruby Thai (First Canadian Place),43.649091,-79.381600,Thai Restaurant
11,First Canadian Place / Underground city,0.020000,1,43.648429,-79.382280,Thai Island,43.649146,-79.383798,Thai Restaurant
29,St. James Town,0.012195,1,43.651494,-79.375418,Sukhothai,43.648487,-79.374547,Thai Restaurant


In [54]:
#Cluster 2

to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Thai Restaurant') ]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,High Park / The Junction South,0.090909,2,43.661608,-79.464763,Silk,43.665291,-79.466238,Thai Restaurant
15,High Park / The Junction South,0.090909,2,43.661608,-79.464763,Isaan Der,43.665311,-79.468078,Thai Restaurant


# CONCLUSION

**Here cluster 0 cluster 2 have less number of Thai restaurant so they can be ideal places for opening the thai restaurant near RUNNYMEDE ROAD, DUNDAS STREET WEST etc**

In cluster 1 there are lots of Thai restaurants so it will be less profitable to open restaurants in that area